In [3]:
## Imports
import os
import sys
import argparse
from collections import deque

import cv2
import numpy as np
import torch

sys.path.append('./')

from data_process.ttnet_video_loader import TTNet_Video_Loader
from models.model_utils import create_model, load_pretrained_model
from config.config import parse_configs
from utils.post_processing import post_processing
from utils.misc import time_synchronized
from easydict import EasyDict as edict
from utils.misc import make_folder

In [4]:
def parse_configs():
    parser = argparse.ArgumentParser(description='TTNet Implementation')
    parser.add_argument('--seed', type=int, default=2020, help='re-produce the results with seed random')
    parser.add_argument('--saved_fn', type=str, default='ttnet', metavar='FN', help='The name using for saving logs, models,...')
    ####################################################################
    ##############     Model configs            ###################
    ####################################################################
    parser.add_argument('-a', '--arch', type=str, default='ttnet', metavar='ARCH', help='The name of the model architecture')
    parser.add_argument('--dropout_p', type=float, default=0.5, metavar='P', help='The dropout probability of the model')
    parser.add_argument('--multitask_learning', action='store_true', help='If true, the weights of different losses will be learnt (train).'
                             'If false, a regular sum of different losses will be applied')
    parser.add_argument('--no_local', action='store_true', help='If true, no local stage for ball detection.')
    parser.add_argument('--no_event', action='store_true', help='If true, no event spotting detection.')
    parser.add_argument('--no_seg', action='store_true', help='If true, no segmentation module.')
    parser.add_argument('--pretrained_path', type=str, default=None, metavar='PATH', help='the path of the pretrained checkpoint')
    parser.add_argument('--overwrite_global_2_local', action='store_true', help='If true, the weights of the local stage will be overwritten by the global stage.')

    ####################################################################
    ##############     Dataloader and Running configs            #######
    ####################################################################
    parser.add_argument('--working-dir', type=str, default='../../', metavar='PATH', help='the ROOT working directory')
    parser.add_argument('--no-val', action='store_true', help='If true, use all data for training, no validation set')
    parser.add_argument('--no-test', action='store_true', help='If true, dont evaluate the model on the test set')
    parser.add_argument('--val-size', type=float, default=0.2, help='The size of validation set')
    parser.add_argument('--smooth-labelling', action='store_true', help='If true, smoothly make the labels of event spotting')
    parser.add_argument('--num_samples', type=int, default=None, help='Take a subset of the dataset to run and debug')
    parser.add_argument('--num_workers', type=int, default=4, help='Number of threads for loading data')
    parser.add_argument('--batch_size', type=int, default=8, help='mini-batch size (default: 8), this is the total'
                             'batch size of all GPUs on the current node when using'
                             'Data Parallel or Distributed Data Parallel')
    parser.add_argument('--print_freq', type=int, default=50, metavar='N', help='print frequency (default: 50)')
    parser.add_argument('--checkpoint_freq', type=int, default=2, metavar='N', help='frequency of saving checkpoints (default: 2)')
    parser.add_argument('--sigma', type=float, default=1., metavar='SIGMA', help='standard deviation of the 1D Gaussian for the ball position target')
    parser.add_argument('--thresh_ball_pos_mask', type=float, default=0.05, metavar='THRESH', help='the lower thresh for the 1D Gaussian of the ball position target')
    ####################################################################
    ##############     Training strategy            ###################
    ####################################################################

    parser.add_argument('--start_epoch', type=int, default=1, metavar='N', help='the starting epoch')
    parser.add_argument('--num_epochs', type=int, default=30, metavar='N', help='number of total epochs to run')
    parser.add_argument('--lr', type=float, default=1e-3, metavar='LR', help='initial learning rate')
    parser.add_argument('--minimum_lr', type=float, default=1e-7, metavar='MIN_LR', help='minimum learning rate during training')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='momentum')
    parser.add_argument('-wd', '--weight_decay', type=float, default=0., metavar='WD', help='weight decay (default: 1e-6)')
    parser.add_argument('--optimizer_type', type=str, default='adam', metavar='OPTIMIZER', help='the type of optimizer, it can be sgd or adam')
    parser.add_argument('--lr_type', type=str, default='plateau', metavar='SCHEDULER', help='the type of the learning rate scheduler (steplr or ReduceonPlateau)')
    parser.add_argument('--lr_factor', type=float, default=0.5, metavar='FACTOR', help='reduce the learning rate with this factor')
    parser.add_argument('--lr_step_size', type=int, default=5, metavar='STEP_SIZE', help='step_size of the learning rate when using steplr scheduler')
    parser.add_argument('--lr_patience', type=int, default=3, metavar='N', help='patience of the learning rate when using ReduceoPlateau scheduler')
    parser.add_argument('--earlystop_patience', type=int, default=None, metavar='N', help='Early stopping the training process if performance is not improved within this value')
    parser.add_argument('--freeze_global', action='store_true', help='If true, no update/train weights for the global stage of ball detection.')
    parser.add_argument('--freeze_local', action='store_true', help='If true, no update/train weights for the local stage of ball detection.')
    parser.add_argument('--freeze_event', action='store_true', help='If true, no update/train weights for the event module.')
    parser.add_argument('--freeze_seg', action='store_true', help='If true, no update/train weights for the segmentation module.')

    ####################################################################
    ##############     Loss weight            ###################
    ####################################################################
    parser.add_argument('--bce_weight', type=float, default=0.5, help='The weight of BCE loss in segmentation module, the dice_loss weight = 1- bce_weight')
    parser.add_argument('--global_weight', type=float, default=1., help='The weight of loss of the global stage for ball detection')
    parser.add_argument('--local_weight', type=float, default=1., help='The weight of loss of the local stage for ball detection')
    parser.add_argument('--event_weight', type=float, default=1., help='The weight of loss of the event spotting module')
    parser.add_argument('--seg_weight', type=float, default=1., help='The weight of BCE loss in segmentation module')

    ####################################################################
    ##############     Distributed Data Parallel            ############
    ####################################################################
    parser.add_argument('--world-size', default=-1, type=int, metavar='N', help='number of nodes for distributed training')
    parser.add_argument('--rank', default=-1, type=int, metavar='N', help='node rank for distributed training')
    parser.add_argument('--dist-url', default='tcp://127.0.0.1:29500', type=str, help='url used to set up distributed training')
    parser.add_argument('--dist-backend', default='nccl', type=str, help='distributed backend')
    parser.add_argument('--gpu_idx', default=0, type=int, help='GPU index to use.')
    parser.add_argument('--no_cuda', action='store_true', help='If true, cuda is not used.')
    parser.add_argument('--multiprocessing-distributed', action='store_true', help='Use multi-processing distributed training to launch '
                             'N processes per node, which has N GPUs. This is the '
                             'fastest way to use PyTorch for either single node or '
                             'multi node data parallel training')
    ####################################################################
    ##############     Evaluation configurations     ###################
    ####################################################################
    parser.add_argument('--evaluate', action='store_true', help='only evaluate the model, not training')
    parser.add_argument('--resume_path', type=str, default=None, metavar='PATH', help='the path of the resumed checkpoint')
    parser.add_argument('--use_best_checkpoint', action='store_true', help='If true, choose the best model on val set, otherwise choose the last model')
    parser.add_argument('--seg_thresh', type=float, default=0.5, help='threshold of the segmentation output')
    parser.add_argument('--event_thresh', type=float, default=0.5, help='threshold of the event spotting output')
    parser.add_argument('--save_test_output', action='store_true', help='If true, the image of testing phase will be saved')

    ####################################################################
    ##############     Demonstration configurations     ###################
    ####################################################################
    parser.add_argument('--video_path', type=str, default=None, metavar='PATH', help='the path of the video that needs to demo')
    parser.add_argument('--output_format', type=str, default='text', metavar='PATH', help='the type of the demo output')
    parser.add_argument('--show_image', action='store_true', help='If true, show the image during demostration')
    parser.add_argument('--save_demo_output', action='store_true', help='If true, the image of demonstration phase will be saved')

    configs = edict(vars(parser.parse_args([])))

    return configs

In [5]:
def config_process(configs):
    ####################################################################
    ############## Hardware configurations ############################
    ####################################################################
    configs.device = torch.device('cpu' if configs.no_cuda else 'cuda')
    configs.ngpus_per_node = torch.cuda.device_count()

    configs.pin_memory = True

    ####################################################################
    ##############     Data configs            ###################
    ####################################################################
    configs.dataset_dir = os.path.join(configs.working_dir, 'dataset')
    configs.train_game_list = ['game_1']#, 'game_2', 'game_3', 'game_4', 'game_5']
    configs.test_game_list = ['test_1']#, 'test_2', 'test_3', 'test_4', 'test_5', 'test_6', 'test_7']
    configs.events_dict = {
        'bounce': 0,
        'net': 1,
        'empty_event': 2
    }
    configs.events_weights_loss_dict = {
        'bounce': 1.,
        'net': 3.,
    }
    configs.events_weights_loss = (configs.events_weights_loss_dict['bounce'], configs.events_weights_loss_dict['net'])
    configs.num_events = len(configs.events_weights_loss_dict)  # Just "bounce" and "net hits"
    configs.num_frames_sequence = 9

    configs.org_size = (1920, 1080)
    configs.input_size = (320, 128)

    configs.tasks = ['global', 'local', 'event', 'seg']
    if configs.no_local:
        if 'local' in configs.tasks:
            configs.tasks.remove('local')
        if 'event' in configs.tasks:
            configs.tasks.remove('event')
    if configs.no_event:
        if 'event' in configs.tasks:
            configs.tasks.remove('event')
    if configs.no_seg:
        if 'seg' in configs.tasks:
            configs.tasks.remove('seg')

    # Compose loss weight for tasks, normalize the weights later
    loss_weight_dict = {
        'global': configs.global_weight,
        'local': configs.local_weight,
        'event': configs.event_weight,
        'seg': configs.seg_weight
    }
    configs.tasks_loss_weight = [loss_weight_dict[task] for task in configs.tasks]

    configs.freeze_modules_list = []
    if configs.freeze_global:
        configs.freeze_modules_list.append('ball_global_stage')
    if configs.freeze_local:
        configs.freeze_modules_list.append('ball_local_stage')
    if configs.freeze_event:
        configs.freeze_modules_list.append('events_spotting')
    if configs.freeze_seg:
        configs.freeze_modules_list.append('segmentation')

    ####################################################################
    ############## logs, Checkpoints, and results dir ########################
    ####################################################################
    configs.checkpoints_dir = os.path.join(configs.working_dir, 'checkpoints', configs.saved_fn)
    configs.logs_dir = os.path.join(configs.working_dir, 'logs', configs.saved_fn)
    configs.use_best_checkpoint = True

    if configs.use_best_checkpoint:
        configs.saved_weight_name = os.path.join(configs.checkpoints_dir, '{}_best.pth'.format(configs.saved_fn))
    else:
        configs.saved_weight_name = os.path.join(configs.checkpoints_dir, '{}.pth'.format(configs.saved_fn))

    configs.results_dir = os.path.join(configs.working_dir, 'results')

    make_folder(configs.checkpoints_dir)
    make_folder(configs.logs_dir)
    make_folder(configs.results_dir)

    if configs.save_test_output:
        configs.saved_dir = os.path.join(configs.results_dir, configs.saved_fn)
        make_folder(configs.saved_dir)

    if configs.save_demo_output:
        configs.save_demo_dir = os.path.join(configs.results_dir, 'demo', configs.saved_fn)
        make_folder(configs.save_demo_dir)

    return configs

In [6]:
configs = parse_configs()

In [7]:
configs = config_process(configs)

In [16]:
configs.video_path       = "/home/stupa/Desktop/Rahul/Research/TTnet/dataset/training/videos/game_1.mp4"
configs.save_demo_dir    = "../results/"
configs.save_demo_output = True

In [11]:
video_loader          = TTNet_Video_Loader(configs.video_path, configs.input_size, configs.num_frames_sequence)
result_filename       = os.path.join(configs.save_demo_dir, 'results.txt')

Length of the video: 88599 frames


In [12]:
frame_rate            = video_loader.video_fps

In [17]:
if configs.save_demo_output:
    configs.frame_dir = os.path.join(configs.save_demo_dir, 'frame')
    if not os.path.isdir(configs.frame_dir):
        os.makedirs(configs.frame_dir)

In [18]:
configs.device = torch.device('cuda:{}'.format(configs.gpu_idx))

In [19]:
# model
model = create_model(configs)
model.cuda()

Unbalance_Loss_Model(
  (model): TTNet(
    (ball_global_stage): BallDetection(
      (conv1): Conv2d(27, 64, kernel_size=(1, 1), stride=(1, 1))
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (convblock1): ConvBlock(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (convblock2): ConvBlock(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (dropout2d): Dropout2d(p=0.5, inplace=False)
      

In [20]:
configs.pretrained_path = "../checkpoints/ttnet_3nd_phase/ttnet_3nd_phase_epoch_30.pth"

In [21]:
assert configs.pretrained_path is not None, "Need to load the pre-trained model"

In [22]:
model = load_pretrained_model(model, configs.pretrained_path, configs.gpu_idx, configs.overwrite_global_2_local)

In [23]:
model.eval()
middle_idx   = int(configs.num_frames_sequence / 2)
queue_frames = deque(maxlen=middle_idx + 1)
frame_idx    = 0
w_original, h_original = 1920, 1080
w_resize, h_resize = 320, 128
w_ratio = w_original / w_resize
h_ratio = h_original / h_resize

In [24]:
# for count, resized_imgs in video_loader:
#     if count % 500 == 0:
#         print(count)

In [25]:
configs.frame_dir

'../results/frame'

In [26]:
def plot_detection(img, ball_pos, seg_img, events):
    """Show the predicted information in the image"""
    img = cv2.addWeighted(img, 1., seg_img * 255, 0.3, 0)
    img = cv2.circle(img, tuple(ball_pos), 5, (255, 0, 255), -1)
    event_name = 'is bounce: {:.2f}, is net: {:.2f}'.format(events[0], events[1])
    img = cv2.putText(img, event_name, (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
    return img

In [27]:
with torch.no_grad():
    for count, resized_imgs in video_loader:
        # take the middle one
        img = cv2.resize(resized_imgs[3 * middle_idx: 3 * (middle_idx + 1)].transpose(1, 2, 0), (w_original, h_original))
        # Expand the first dim
        resized_imgs = torch.from_numpy(resized_imgs).to(configs.device, non_blocking=True).float().unsqueeze(0)
        t1 = time_synchronized()
        pred_ball_global, pred_ball_local, pred_events, pred_seg = model.run_demo(resized_imgs)
        t2 = time_synchronized()
        prediction_global, prediction_local, prediction_seg, prediction_events = post_processing(
            pred_ball_global, pred_ball_local, pred_events, pred_seg, configs.input_size[0],
            configs.thresh_ball_pos_mask, configs.seg_thresh, configs.event_thresh)
        prediction_ball_final = [
            int(prediction_global[0] * w_ratio + prediction_local[0] - w_resize / 2),
            int(prediction_global[1] * h_ratio + prediction_local[1] - h_resize / 2)
        ]

        # Get infor of the (middle_idx + 1)th frame
        if len(queue_frames) == middle_idx + 1:
            frame_pred_infor = queue_frames.popleft()
            seg_img = frame_pred_infor['seg'].astype(np.uint8)
            ball_pos = frame_pred_infor['ball']
            seg_img = cv2.resize(seg_img, (w_original, h_original))
            ploted_img = plot_detection(img, ball_pos, seg_img, prediction_events)

            ploted_img = cv2.cvtColor(ploted_img, cv2.COLOR_RGB2BGR)
            if configs.show_image:
                cv2.imshow('ploted_img', ploted_img)
                cv2.waitKey(10)
            if configs.save_demo_output:
                cv2.imwrite(os.path.join(configs.frame_dir, '{:06d}.jpg'.format(frame_idx)), ploted_img)

        frame_pred_infor = {
            'seg': prediction_seg,
            'ball': prediction_ball_final
        }
        queue_frames.append(frame_pred_infor)

        frame_idx += 1
        print('Done frame_idx {} - time {:.3f}s'.format(frame_idx, t2 - t1))



Done frame_idx 1 - time 0.388s
Done frame_idx 2 - time 0.005s
Done frame_idx 3 - time 0.005s
Done frame_idx 4 - time 0.005s
Done frame_idx 5 - time 0.005s
Done frame_idx 6 - time 0.005s
Done frame_idx 7 - time 0.005s
Done frame_idx 8 - time 0.005s
Done frame_idx 9 - time 0.005s
Done frame_idx 10 - time 0.006s
Done frame_idx 11 - time 0.006s
Done frame_idx 12 - time 0.005s
Done frame_idx 13 - time 0.006s
Done frame_idx 14 - time 0.007s
Done frame_idx 15 - time 0.005s
Done frame_idx 16 - time 0.006s
Done frame_idx 17 - time 0.005s
Done frame_idx 18 - time 0.005s
Done frame_idx 19 - time 0.005s
Done frame_idx 20 - time 0.005s
Done frame_idx 21 - time 0.005s
Done frame_idx 22 - time 0.005s
Done frame_idx 23 - time 0.005s
Done frame_idx 24 - time 0.006s
Done frame_idx 25 - time 0.005s
Done frame_idx 26 - time 0.006s
Done frame_idx 27 - time 0.006s
Done frame_idx 28 - time 0.005s
Done frame_idx 29 - time 0.005s
Done frame_idx 30 - time 0.005s
Done frame_idx 31 - time 0.005s
Done frame_idx 32

Done frame_idx 256 - time 0.005s
Done frame_idx 257 - time 0.005s
Done frame_idx 258 - time 0.005s
Done frame_idx 259 - time 0.005s
Done frame_idx 260 - time 0.005s
Done frame_idx 261 - time 0.005s
Done frame_idx 262 - time 0.005s
Done frame_idx 263 - time 0.005s
Done frame_idx 264 - time 0.005s
Done frame_idx 265 - time 0.005s
Done frame_idx 266 - time 0.006s
Done frame_idx 267 - time 0.005s
Done frame_idx 268 - time 0.005s
Done frame_idx 269 - time 0.005s
Done frame_idx 270 - time 0.005s
Done frame_idx 271 - time 0.005s
Done frame_idx 272 - time 0.005s
Done frame_idx 273 - time 0.005s
Done frame_idx 274 - time 0.006s
Done frame_idx 275 - time 0.006s
Done frame_idx 276 - time 0.005s
Done frame_idx 277 - time 0.005s
Done frame_idx 278 - time 0.005s
Done frame_idx 279 - time 0.005s
Done frame_idx 280 - time 0.005s
Done frame_idx 281 - time 0.005s
Done frame_idx 282 - time 0.007s
Done frame_idx 283 - time 0.005s
Done frame_idx 284 - time 0.005s
Done frame_idx 285 - time 0.005s
Done frame

Done frame_idx 508 - time 0.005s
Done frame_idx 509 - time 0.005s
Done frame_idx 510 - time 0.005s
Done frame_idx 511 - time 0.005s
Done frame_idx 512 - time 0.005s
Done frame_idx 513 - time 0.006s
Done frame_idx 514 - time 0.006s
Done frame_idx 515 - time 0.005s
Done frame_idx 516 - time 0.005s
Done frame_idx 517 - time 0.005s
Done frame_idx 518 - time 0.005s
Done frame_idx 519 - time 0.006s
Done frame_idx 520 - time 0.006s
Done frame_idx 521 - time 0.005s
Done frame_idx 522 - time 0.005s
Done frame_idx 523 - time 0.005s
Done frame_idx 524 - time 0.005s
Done frame_idx 525 - time 0.005s
Done frame_idx 526 - time 0.005s
Done frame_idx 527 - time 0.005s
Done frame_idx 528 - time 0.005s
Done frame_idx 529 - time 0.006s
Done frame_idx 530 - time 0.005s
Done frame_idx 531 - time 0.005s
Done frame_idx 532 - time 0.005s
Done frame_idx 533 - time 0.006s
Done frame_idx 534 - time 0.006s
Done frame_idx 535 - time 0.006s
Done frame_idx 536 - time 0.006s
Done frame_idx 537 - time 0.005s
Done frame

Done frame_idx 760 - time 0.005s
Done frame_idx 761 - time 0.006s
Done frame_idx 762 - time 0.007s
Done frame_idx 763 - time 0.005s
Done frame_idx 764 - time 0.005s
Done frame_idx 765 - time 0.006s
Done frame_idx 766 - time 0.005s
Done frame_idx 767 - time 0.005s
Done frame_idx 768 - time 0.005s
Done frame_idx 769 - time 0.006s
Done frame_idx 770 - time 0.006s
Done frame_idx 771 - time 0.006s
Done frame_idx 772 - time 0.005s
Done frame_idx 773 - time 0.005s
Done frame_idx 774 - time 0.006s
Done frame_idx 775 - time 0.006s
Done frame_idx 776 - time 0.005s
Done frame_idx 777 - time 0.006s
Done frame_idx 778 - time 0.005s
Done frame_idx 779 - time 0.005s
Done frame_idx 780 - time 0.008s
Done frame_idx 781 - time 0.005s
Done frame_idx 782 - time 0.006s
Done frame_idx 783 - time 0.005s
Done frame_idx 784 - time 0.006s
Done frame_idx 785 - time 0.006s
Done frame_idx 786 - time 0.007s
Done frame_idx 787 - time 0.006s
Done frame_idx 788 - time 0.005s
Done frame_idx 789 - time 0.006s
Done frame

Done frame_idx 1012 - time 0.006s
Done frame_idx 1013 - time 0.007s
Done frame_idx 1014 - time 0.007s
Done frame_idx 1015 - time 0.006s
Done frame_idx 1016 - time 0.007s
Done frame_idx 1017 - time 0.007s
Done frame_idx 1018 - time 0.007s
Done frame_idx 1019 - time 0.006s
Done frame_idx 1020 - time 0.006s
Done frame_idx 1021 - time 0.006s
Done frame_idx 1022 - time 0.006s
Done frame_idx 1023 - time 0.006s
Done frame_idx 1024 - time 0.006s
Done frame_idx 1025 - time 0.007s
Done frame_idx 1026 - time 0.007s
Done frame_idx 1027 - time 0.007s
Done frame_idx 1028 - time 0.007s
Done frame_idx 1029 - time 0.006s
Done frame_idx 1030 - time 0.008s
Done frame_idx 1031 - time 0.006s
Done frame_idx 1032 - time 0.006s
Done frame_idx 1033 - time 0.006s
Done frame_idx 1034 - time 0.005s
Done frame_idx 1035 - time 0.005s
Done frame_idx 1036 - time 0.005s
Done frame_idx 1037 - time 0.005s
Done frame_idx 1038 - time 0.005s
Done frame_idx 1039 - time 0.005s
Done frame_idx 1040 - time 0.008s
Done frame_idx

...

Done frame_idx 18963 - time 0.006s
Done frame_idx 18964 - time 0.005s
Done frame_idx 18965 - time 0.006s
Done frame_idx 18966 - time 0.005s
Done frame_idx 18967 - time 0.008s
Done frame_idx 18968 - time 0.007s
Done frame_idx 18969 - time 0.006s
Done frame_idx 18970 - time 0.006s
Done frame_idx 18971 - time 0.006s
Done frame_idx 18972 - time 0.006s
Done frame_idx 18973 - time 0.006s
Done frame_idx 18974 - time 0.006s
Done frame_idx 18975 - time 0.006s
Done frame_idx 18976 - time 0.006s
Done frame_idx 18977 - time 0.007s
Done frame_idx 18978 - time 0.006s
Done frame_idx 18979 - time 0.006s
Done frame_idx 18980 - time 0.008s
Done frame_idx 18981 - time 0.006s
Done frame_idx 18982 - time 0.006s
Done frame_idx 18983 - time 0.006s
Done frame_idx 18984 - time 0.006s
Done frame_idx 18985 - time 0.006s
Done frame_idx 18986 - time 0.005s
Done frame_idx 18987 - time 0.005s
Done frame_idx 18988 - time 0.005s
Done frame_idx 18989 - time 0.005s
Done frame_idx 18990 - time 0.005s
Done frame_idx 18991

Done frame_idx 19203 - time 0.007s
Done frame_idx 19204 - time 0.007s
Done frame_idx 19205 - time 0.007s
Done frame_idx 19206 - time 0.007s
Done frame_idx 19207 - time 0.007s
Done frame_idx 19208 - time 0.007s
Done frame_idx 19209 - time 0.007s
Done frame_idx 19210 - time 0.007s
Done frame_idx 19211 - time 0.007s
Done frame_idx 19212 - time 0.008s
Done frame_idx 19213 - time 0.007s
Done frame_idx 19214 - time 0.007s
Done frame_idx 19215 - time 0.006s
Done frame_idx 19216 - time 0.007s
Done frame_idx 19217 - time 0.006s
Done frame_idx 19218 - time 0.007s
Done frame_idx 19219 - time 0.006s
Done frame_idx 19220 - time 0.007s
Done frame_idx 19221 - time 0.007s
Done frame_idx 19222 - time 0.005s
Done frame_idx 19223 - time 0.005s
Done frame_idx 19224 - time 0.005s
Done frame_idx 19225 - time 0.006s
Done frame_idx 19226 - time 0.005s
Done frame_idx 19227 - time 0.005s
Done frame_idx 19228 - time 0.008s
Done frame_idx 19229 - time 0.009s
Done frame_idx 19230 - time 0.007s
Done frame_idx 19231

Done frame_idx 19443 - time 0.007s
Done frame_idx 19444 - time 0.007s
Done frame_idx 19445 - time 0.005s
Done frame_idx 19446 - time 0.005s
Done frame_idx 19447 - time 0.005s
Done frame_idx 19448 - time 0.005s
Done frame_idx 19449 - time 0.005s
Done frame_idx 19450 - time 0.006s
Done frame_idx 19451 - time 0.008s
Done frame_idx 19452 - time 0.007s
Done frame_idx 19453 - time 0.006s
Done frame_idx 19454 - time 0.006s
Done frame_idx 19455 - time 0.006s
Done frame_idx 19456 - time 0.006s
Done frame_idx 19457 - time 0.006s
Done frame_idx 19458 - time 0.006s
Done frame_idx 19459 - time 0.006s
Done frame_idx 19460 - time 0.005s
Done frame_idx 19461 - time 0.007s
Done frame_idx 19462 - time 0.007s
Done frame_idx 19463 - time 0.006s
Done frame_idx 19464 - time 0.006s
Done frame_idx 19465 - time 0.006s
Done frame_idx 19466 - time 0.006s
Done frame_idx 19467 - time 0.006s
Done frame_idx 19468 - time 0.008s
Done frame_idx 19469 - time 0.006s
Done frame_idx 19470 - time 0.007s
Done frame_idx 19471

Done frame_idx 19683 - time 0.008s
Done frame_idx 19684 - time 0.006s
Done frame_idx 19685 - time 0.005s
Done frame_idx 19686 - time 0.006s
Done frame_idx 19687 - time 0.006s
Done frame_idx 19688 - time 0.005s
Done frame_idx 19689 - time 0.005s
Done frame_idx 19690 - time 0.005s
Done frame_idx 19691 - time 0.005s
Done frame_idx 19692 - time 0.008s
Done frame_idx 19693 - time 0.007s
Done frame_idx 19694 - time 0.006s
Done frame_idx 19695 - time 0.007s
Done frame_idx 19696 - time 0.006s
Done frame_idx 19697 - time 0.005s
Done frame_idx 19698 - time 0.005s
Done frame_idx 19699 - time 0.005s
Done frame_idx 19700 - time 0.005s
Done frame_idx 19701 - time 0.005s
Done frame_idx 19702 - time 0.007s
Done frame_idx 19703 - time 0.009s
Done frame_idx 19704 - time 0.006s
Done frame_idx 19705 - time 0.009s
Done frame_idx 19706 - time 0.006s
Done frame_idx 19707 - time 0.006s
Done frame_idx 19708 - time 0.006s
Done frame_idx 19709 - time 0.005s
Done frame_idx 19710 - time 0.005s
Done frame_idx 19711

KeyboardInterrupt: 

In [31]:
if configs.output_format == 'video':
    output_video_path = os.path.join(configs.save_demo_dir, 'result.mp4')
    cmd_str = 'ffmpeg -f image2 -i {}/%05d.jpg -b 5000k -c:v mpeg4 {}'.format(
        os.path.join(configs.frame_dir), output_video_path)
    os.system(cmd_str)

#  ffmpeg -framerate 30 -pattern_type glob -i 'frame/*.jpg'  -c:v libx264 -pix_fmt yuv420p out.mp4




In [30]:
configs.output_format = "video"

In [32]:
configs.save_demo_dir

'../results/'

In [36]:
 os.path.join(configs.frame_dir, output_video_path)

'../results/frame/../results/result.mp4'